In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from whitebox.eval import WhiteBoxSensitivity
from whitebox.eval import WhiteBoxError


In [3]:
df = pd.read_csv("./Test_data.csv")

In [4]:

# set up y var
# set up some params
ydepend = 'response'


df['column1bin'] = df['column1'].apply(lambda x: 'bin_0' if x > 0 else 'bin_1')
df['column5bin'] = df['column5'].apply(lambda x: 'bin_0' if x > 0 else 'bin_1') 

string_categories = df.select_dtypes(include = ['O'])
# iterate over string categories
for cat in string_categories:
    df[cat] = pd.Categorical(df[cat]) 
    
df.select_dtypes(include = ['category'])
model_df = df.copy(deep = True)

# create dummies example using all categorical columns
dummies = pd.concat([pd.get_dummies(model_df.loc[:, col], prefix = col) for col in model_df.select_dtypes(include = ['category']).columns], axis = 1)
finaldf = pd.concat([model_df.select_dtypes(include = [np.number]), dummies], axis = 1)


In [6]:
#from sklearn.linear_model import Lasso
#clf = Lasso(alpha = 0.1)

#from sklearn.linear_model import Lars
#clf = Lars()

from sklearn.linear_model import MultiTaskLasso
clf = MultiTaskLasso()

# fit the model using the dummy dataframe
clf.fit(finaldf.loc[:, finaldf.columns != ydepend], df.loc[:, ydepend])

start=datetime.now()
print start
WB = WhiteBoxError(clf,
                    model_df=finaldf,
                    ydepend=ydepend,
                    cat_df=df,
                    featuredict=None,
                    groupbyvars=['column1bin','column5bin'],
                    aggregate_func=np.mean,
                    verbose=None
                    )
WB.run()

WB.save('./multiTaskLasso-Error.html')

print "RUN TIME----->", datetime.now()-start

2018-02-16 14:16:09.512000
C:\Users\shrupanda\AnacondaProjects\Whitebox
C:\Users\shrupanda\AnacondaProjects\Whitebox
RUN TIME-----> 0:55:26.372000


In [7]:

#from sklearn.linear_model import Lasso
#clf = Lasso(alpha = 0.1)

#from sklearn.linear_model import Lars
#clf = Lars()

from sklearn.linear_model import MultiTaskLasso
clf = MultiTaskLasso()

# fit the model using the dummy dataframe
clf.fit(finaldf.loc[:, finaldf.columns != ydepend], df.loc[:, ydepend])

start=datetime.now()
print start
WB = WhiteBoxSensitivity(clf,
                    model_df=finaldf,
                    ydepend=ydepend,
                    cat_df=df,
                    featuredict=None,
                    groupbyvars=['column1bin','column5bin'],
                    aggregate_func=np.mean,
                    verbose=None,
                     std_num=2
                    )
WB.run()

WB.save('./multiTaskLasso-Sensitivity.html')

print "RUN TIME----->", datetime.now()-start

ValueError: For mono-task outputs, use ElasticNet